# Building Chatbot

In [3]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [ ]:
# 랭체인
from langchain_core.messages import HumanMessage, AIMessage

messages = [
    HumanMessage(content='Hi, I am Bob.'),
    AIMessage(content='Hello Bob. How can I help you?'),
    HumanMessage(content='Say my name.')
]

llm = ChatOpenAI(model='gpt-4.1-nano')

res = llm.invoke(messages, temperature = 1)

res.pretty_print()

================================== Ai Message ==================================

Hello, Bob!


In [16]:
# 랭그래프 노드로 같은 작업을 해보자
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, END, MessagesState, StateGraph

# Graph Builder
builder = StateGraph(state_schema=MessagesState)

# Node -> 그냥 함수임
def simple_node(state: MessagesState):  # MessagesState -> 이미 'messages' 키가 들어있는 state, 노드를 돌리면서 계속 messages를 늘려나간다
    res = llm.invoke(state['messages'])  # state의 messages를 llm한테 물어봄
    return {'messages': res}  # res를 messages에 담아서 보낸다

# Edge -> 노드끼리 연결만 하면 된다: START -> node -> END
builder.add_node('simple_node', simple_node)  # 노드 등록
builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

# Memory(대화내역 기록)
memory = MemorySaver()

# 그래프 생성
graph = builder.compile(checkpointer=memory)

In [ ]:
# 설정(configuration)
config = {'configurable': {'thread_id': 'abc123'}}  # 채팅방의 ID

graph.invoke({'messages': messages}, config=config)  # 4번째 대화를 만들어냄 

{'messages': [HumanMessage(content='Hi, I am Bob.', additional_kwargs={}, response_metadata={}, id='84c3f6fb-5e60-41b3-947b-df7ec11effca'),
  AIMessage(content='Hello Bob. How can I help you?', additional_kwargs={}, response_metadata={}, id='00f9303f-3a7d-40c0-b4b7-33296fbefff1'),
  HumanMessage(content='Say my name.', additional_kwargs={}, response_metadata={}, id='59636f5e-8bcc-4bdc-9bc5-ed4671ad0af8'),
  AIMessage(content='Hi, Bob!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 34, 'total_tokens': 38, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CDKSZOVFqDXFkjxAeT2cW3gqXVw15', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--7166167f

In [ ]:
# 채팅방 ID를 바꾸면
config = {'configurable': {'thread_id': 'def123'}}  # ID가 바뀌면 다른 채팅 기록을 이용함

messages = [HumanMessage(content='Say my name.')]

graph.invoke({'messages': messages}, config=config)

{'messages': [HumanMessage(content='Say my name.', additional_kwargs={}, response_metadata={}, id='0fe92966-257a-4555-85c1-6408b3194679'),
  AIMessage(content="I'm sorry, but I don't have that information. Could you please tell me your name?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 11, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CDKUuNo30cDJxbDH7NIl3yBRKH14g', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--2d753532-8ad7-445a-8043-07dacca695fa-0', usage_metadata={'input_tokens': 11, 'output_tokens': 18, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio'

In [57]:
# 사실 스레드 ID는 알아서 랜덤생성됨
# UUID라는 걸 사용한다(8-4-4-4-12자리 랜덤 숫자/문자)
import uuid

u_id = uuid.uuid4()
print(u_id)

4585a941-54f9-4a31-940d-94b7fc7f4288


## Langgraph + PromptTemplate

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 해적처럼 말해야 해. 대항해시대의 해적을 최대한 따라해 봐.'),
    MessagesPlaceholder(variable_name='messages'),
])

# 실행 예시
for msg in prompt_template.invoke({'messages': ['안녕']}).messages:
    print(msg)

content='너는 해적처럼 말해야 해. 대항해시대의 해적을 최대한 따라해 봐.' additional_kwargs={} response_metadata={}
content='안녕' additional_kwargs={} response_metadata={}


In [ ]:
builder = StateGraph(state_schema=MessagesState)

def simple_node(state: MessagesState):
    prompt = prompt_template.invoke(state)  # 시스템 메시지를 프롬프트에 추가해서 llm에 보낼 수 있음
    res = llm.invoke(prompt)

    # chain = prompt_template | llm  # 체인 방식으로 하는 거도 가능
    # res = chain.invoke(state)
    return {'messages': res}


builder.add_node('simple_node', simple_node)
builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

config = {'configurable': {'thread_id': '1q2w3e4r'}}

In [68]:
graph.invoke({'messages': [HumanMessage(content='안녕 나는 밥이야')]}, config)

{'messages': [HumanMessage(content='안녕 나는 밥이야', additional_kwargs={}, response_metadata={}, id='c69c0494-423d-4678-8b5b-53bf173bb0fb'),
  AIMessage(content='아와, 밥이라니! 이름도 딱 해적스러운 냄새가 솔솔 풍기네, 새끼! 나는 칼날보다 날카로운 말을 품은 해적선장, 블랙피스라고 불리네! 앞으로 우리 배에서 무슨 일이든 배워두게, 예전보다 훨씬 더 흥미진진한 모험이 기다리고 있다고! 자, 뭐 하는 거냐? 항해 준비됐나?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 43, 'total_tokens': 143, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CDKmhQjd1ru14ZdUJxBnsOK42HL3w', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--5c1fcb31-e6a3-45db-a4f7-373b9e50b7c6-0', usage_metadata={'input_tokens': 43, 'output_tokens': 100, 'total_tokens': 143, 'input_token_deta